In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, KBinsDiscretizer
from deep_logic.models.brl import XBRLClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader
import time
from sklearn.metrics import accuracy_score
from sympy import simplify_logic
from sklearn.model_selection import train_test_split

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relu_nn import get_reduced_model, prune_features
from deep_logic.utils.psi_nn import prune_equal_fanin
from deep_logic import logic
import deep_logic as dl

set_seed(0)

#%%

data = pd.read_csv('data/mimic-ii/full_cohort_data.csv')
# data.drop('hgb_first')
fs = [
    'aline_flg',
    'gender_num',
    # 'hosp_exp_flg',
    # 'icu_exp_flg',
    # 'day_28_flg',
    # 'censor_flg',
    'sepsis_flg', 'chf_flg', 'afib_flg',
    'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg',
    'mal_flg', 'resp_flg',
]
features = fs
data1 = data[fs].values
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
data1 = imp_mean.fit_transform(data1)

f2 = fs.copy()
f2.append('day_icu_intime')
f2.append('service_unit')
f2.append('day_28_flg')
f2.append('hospital_los_day')
f2.append('icu_exp_flg')
f2.append('hosp_exp_flg')
f2.append('censor_flg')
f2.append('mort_day_censored')
f2 = data.columns.difference(f2)
data2 = data[f2].values
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
data2 = imp_mean.fit_transform(data2)
scaler = MinMaxScaler((0, 1))
data2 = scaler.fit_transform(data2)
features = features + list(f2)
est = KBinsDiscretizer(n_bins=3, encode='onehot-dense', strategy='uniform')
data2d = est.fit_transform(data2)
f2d = []
for feature in f2:
    #f2d.append(feature + '_VLOW')
    f2d.append(feature + '_LOW')
    f2d.append(feature + '_NORMAL')
    f2d.append(feature + '_HIGH')
    #f2d.append(feature + '_VHIGH')
features = fs + f2d

datax = np.hstack((data1, data2d))
datay = data['day_28_flg'].values

x = torch.FloatTensor(datax)
y = torch.LongTensor(datay)
print(x.shape)
print(y.shape)


torch.Size([1776, 90])
torch.Size([1776])


In [2]:
results_dir = 'results/mimic2'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# concepts = [f'c{i:03}' for i in range(x.shape[1])]
concepts =  features
n_rep = 10
tot_epochs = 5001
prune_epochs = 2001
seed = 42

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [3]:
def train_nn(x_train, y_train, need_pruning, seed, device, l1=0.001, lr=0.001, relu=False, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 100),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 10),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)
        
        if need_pruning or relu:
            for module in model.children():
                if isinstance(module, torch.nn.Linear):
                    loss += l1 * torch.norm(module.weight, 1)
                    loss += l1 * torch.norm(module.bias, 1)
                    break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning and not relu and epoch % 1000 == 1:
            prune_features(model, n_classes=1, device=device)
            need_pruning = True
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [4]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [16]:
def c_to_y(method, need_pruning, l1=0.0001, lr=0.0001, relu=False, verbose=False):
    methods = []
    splits = []
    explanations = []
    explanations_inv = []
    model_accuracies = []
    explanation_accuracies = []
    explanation_accuracies_inv = []
    elapsed_times = []
    elapsed_times_inv = []
    explanation_fidelities = []
    explanation_complexities = []
    for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.cpu().detach().numpy())):
        print(f'Split [{split+1}/{n_splits}]')
        x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
        y_trainval, y_test = torch.LongTensor(y[trainval_index]), torch.LongTensor(y[test_index])
        x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.3, random_state=42)
        
        y_preds_trainval = y_trainval
        y_preds_train = y_train
        y_preds_val = y_val
        y_preds_test = y_test
    
        explanation, explanation_inv = '', ''
        explanation_accuracy, explanation_accuracy_inv = 0, 0
        
        if method == 'tree':
            classifier = DecisionTreeClassifier(random_state=seed)
            classifier.fit(x_trainval.cpu().detach().numpy(), y_preds_trainval.cpu().detach().numpy())
            y_preds = classifier.predict(x_test.cpu().detach().numpy())
            model_accuracy = accuracy_score(y_preds, y_test)

            target_class = 1
            start = time.time()
            explanation = tree_to_formula(classifier, concepts, target_class)
            elapsed_time = time.time() - start
            explanation_accuracy = model_accuracy
            explanation_fidelity = accuracy_score(y_preds_test.cpu().detach().numpy(), y_preds)
            explanation_complexity = dl.logic.complexity(explanation)
        
        elif method == 'brl':
            y_preds_trainval2 = torch.zeros((y_preds_trainval.shape[0], 2))
            print(y_preds_trainval2.shape)
            y_preds_trainval2[:, 0] = 1-y_preds_trainval
            y_preds_trainval2[:, 1] = y_preds_trainval
            train_dataset = TensorDataset(x_trainval, y_preds_trainval2)
            y_preds_val2 = torch.zeros((y_preds_val.shape[0], 2))
            print(y_preds_val2.shape)
            y_preds_val2[:, 0] = 1-y_preds_val
            y_preds_val2[:, 1] = y_preds_val
            val_dataset = TensorDataset(x_val, y_preds_val2)
            test_dataset = TensorDataset(x_test, y_preds_test)
            dummy_concepts = [f'feature{i:010}' for i in range(len(concepts))]
            model = XBRLClassifier(name=os.path.join(results_dir, f'{method}_{split}'), 
                                   n_classes=len(y_train.unique()), 
                                   n_features=x_train.shape[1], feature_names=dummy_concepts, 
                                   class_names=['~day_28_flg', 'day_28_flg'], discretize=True)
            results = model.fit(train_dataset, metric=accuracy_score, save=True)
            model_accuracy = model.evaluate(test_dataset)
            target_class = 1
            global_explanation, elapsed_time = model.get_global_explanation(target_class, concepts=dummy_concepts, return_time=True)
            explanation_fidelity = 0
            explanation_complexity = 0
            if global_explanation:
                explanation = logic.base.replace_names(global_explanation, concepts)
                explanation_accuracy, y_formula = logic.base.test_explanation(global_explanation, 
                                                                              target_class, 
                                                                              x=x_test, y=y_test,
                                                                              metric=accuracy_score)
                explanation_fidelity = dl.logic.fidelity(y_formula, y_preds_test)
                explanation_complexity = dl.logic.complexity(global_explanation)
                
        else:
            if method == 'psi':
                # positive class
                target_class = 1
                model = train_psi_nn(x_trainval, y_preds_trainval.eq(target_class), need_pruning, seed, device, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()>0.5
                model_accuracy = accuracy_score(y_preds_test.cpu().detach().numpy(), y_preds)
                
            else:
                model = train_nn(x_trainval, y_preds_trainval, need_pruning, seed, device, l1, lr, relu, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy().argmax(axis=1)
                model_accuracy = accuracy_score(y_preds_test.cpu().detach().numpy(), y_preds)

            # positive class
            target_class = 1
            start = time.time()
            if method == 'psi':
                global_explanation = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                                   x_val.to(device), 
                                                                                   y_preds_val.to(device), 
                                                                                   topk_explanations=2,
                                                                                   target_class=target_class,
                                                                                   method=method, device=device)
            elapsed_time = time.time() - start
            
            explanation_fidelity = 0
            explanation_complexity = 0
            if global_explanation:
                explanation = logic.base.replace_names(global_explanation, concepts)
                explanation_accuracy, y_formula = logic.base.test_explanation(global_explanation, 
                                                                              target_class, 
                                                                              x=x_test, y=y_test,
                                                                              metric=accuracy_score)
                explanation_fidelity = dl.logic.fidelity(y_formula, y_preds_test)
                explanation_complexity = dl.logic.complexity(global_explanation)

        if verbose:
            print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
            print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
            print(f'\t Elapsed time {elapsed_time}')
            print(f'\t Fidelity: "{explanation_fidelity:.4f}" - Complexity: "{explanation_complexity}"')

        methods.append(method)
        splits.append(split)
        explanations.append(explanation)
        model_accuracies.append(model_accuracy)
        explanation_accuracies.append(explanation_accuracy)
        elapsed_times.append(elapsed_time)
        explanation_fidelities.append(explanation_fidelity)
        explanation_complexities.append(explanation_complexity)
    
    explanation_consistency = dl.logic.formula_consistency(explanations)
    print(f'Consistency of explanations: {explanation_consistency:.4f}')
    
    results = pd.DataFrame({
        'method': methods,
        'split': splits,
        'explanation': explanations,
        'model_accuracy': model_accuracies,
        'explanation_accuracy': explanation_accuracies,
        'explanation_fidelity': explanation_fidelities,
        'explanation_complexity': explanation_complexities,
        'explanation_consistency': explanation_consistency,
        'elapsed_time': elapsed_times,
    })
    results.to_csv(os.path.join(results_dir, f'results_{method}.csv'))
    
    return results

# General pruning

In [6]:
method = 'pruning'
need_pruning = True
relu = False
results_pruning = c_to_y(method, need_pruning, relu=relu)
results_pruning

Split [1/10]
Split [2/10]
Split [3/10]
Split [4/10]
Split [5/10]
Split [6/10]
Split [7/10]
Split [8/10]
Split [9/10]
Split [10/10]
Consistency of explanations: 0.4400


,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,pruning,0,stroke_flg & ~sapsi_first_LOW & ~weight_first_...,0.853933,0.853933,0.853933,3,0.44,0.051862
1,pruning,1,stroke_flg & ~sapsi_first_LOW,0.848315,0.848315,0.848315,2,0.44,0.089762
2,pruning,2,stroke_flg & age_HIGH & ~hgb_first_LOW & ~saps...,0.876404,0.876404,0.876404,4,0.44,0.057845
3,pruning,3,stroke_flg & ~sapsi_first_LOW,0.859551,0.859551,0.859551,2,0.44,0.072805
4,pruning,4,stroke_flg & age_HIGH & ~sapsi_first_LOW,0.859551,0.859551,0.859551,3,0.44,0.077791
5,pruning,5,stroke_flg & ~sapsi_first_LOW,0.837079,0.837079,0.837079,2,0.44,0.043883
6,pruning,6,stroke_flg & ~sapsi_first_LOW,0.830508,0.830508,0.830508,2,0.44,0.061836
7,pruning,7,stroke_flg,0.858757,0.858757,0.858757,1,0.44,0.049866
8,pruning,8,stroke_flg,0.847458,0.847458,0.847458,1,0.44,0.094778
9,pruning,9,stroke_flg & age_HIGH,0.870056,0.870056,0.870056,2,0.44,0.046907


# ReLUs

In [8]:
method = 'weights'
need_pruning = False
relu = True
results_weights = c_to_y(method='weights', need_pruning=False, l1=0.005, lr=0.0005, relu=True, verbose=True)
results_weights

Split [1/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.8411
	 Epoch 1500: train accuracy: 0.8411
	 Epoch 2000: train accuracy: 0.8411
	 Epoch 2500: train accuracy: 0.8411
	 Epoch 3000: train accuracy: 0.8411
	 Epoch 3500: train accuracy: 0.8817
	 Epoch 4000: train accuracy: 0.8899
	 Epoch 4500: train accuracy: 0.9036
	 Epoch 5000: train accuracy: 0.9118
	 Model's accuracy: 0.8708
	 Class 1 - Global explanation: "(~stroke_flg & ~age_NORMAL) | (bun_first_LOW & ~age_LOW & ~sapsi_first_LOW & ~weight_first_NORMAL)" - Accuracy: 0.3090
	 Elapsed time 0.19148802757263184
	 Fidelity: "0.3090" - Complexity: "6"
Split [2/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.8411
	 Epoch 1500: train accuracy: 0.8411
	 Epoch 2000: train accuracy: 0.8411
	 Epoch 2500: train accuracy: 0.8411
	 Epoch 3000: train accuracy: 0.8411
	 Epoch 3500: train accuracy: 0.8830
	 Epoch 4000: train accura

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,weights,0,(~stroke_flg & ~age_NORMAL) | (bun_first_LOW &...,0.870787,0.308989,0.308989,6,0.314286,0.191488
1,weights,1,sofa_first_NORMAL | (abg_count_LOW & age_HIGH ...,0.853933,0.533708,0.533708,4,0.314286,0.288256
2,weights,2,(bun_first_LOW & ~mal_flg & ~sapsi_first_HIGH ...,0.865169,0.280899,0.280899,9,0.314286,0.272273
3,weights,3,bun_first_LOW & ~weight_first_NORMAL,0.887640,0.179775,0.179775,2,0.314286,0.152592
4,weights,4,stroke_flg | ~stroke_flg,0.837079,0.157303,0.157303,2,0.314286,0.190491
5,weights,5,(bun_first_LOW & ~stroke_flg) | (bun_first_LOW...,0.865169,0.202247,0.202247,5,0.314286,0.147605
6,weights,6,~stroke_flg | (bun_first_LOW & ~age_LOW & ~hgb...,0.830508,0.152542,0.152542,5,0.314286,0.206449
7,weights,7,(abg_count_LOW & age_HIGH & bun_first_LOW & ~s...,0.853107,0.757062,0.757062,10,0.314286,0.243351
8,weights,8,(age_HIGH & bun_first_LOW & ~stroke_flg) | (bu...,0.836158,0.542373,0.542373,7,0.314286,0.209441
9,weights,9,bun_first_LOW | ~stroke_flg,0.864407,0.158192,0.158192,2,0.314286,0.167552


# Psi network

In [9]:
method = 'psi'
need_pruning = True
relu = False
results_psi = c_to_y(method, need_pruning, relu, verbose=True)
results_psi

Split [1/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.8411
	 Epoch 1500: train accuracy: 0.8529
	 Epoch 2000: train accuracy: 0.8867
	 Epoch 2500: train accuracy: 0.8411
	 Epoch 3000: train accuracy: 0.8411
	 Epoch 3500: train accuracy: 0.8411
	 Epoch 4000: train accuracy: 0.8411
	 Epoch 4500: train accuracy: 0.8411
	 Epoch 5000: train accuracy: 0.8411
	 Model's accuracy: 0.8371
	 Class 1 - Global explanation: "(stroke_flg | bun_first_NORMAL | sapsi_first_HIGH)" - Accuracy: 0.8315
	 Elapsed time 0.0359039306640625
	 Fidelity: "0.8315" - Complexity: "3"
Split [2/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.8411
	 Epoch 1500: train accuracy: 0.8586
	 Epoch 2000: train accuracy: 0.8899
	 Epoch 2500: train accuracy: 0.8411
	 Epoch 3000: train accuracy: 0.8411
	 Epoch 3500: train accuracy: 0.8411
	 Epoch 4000: train accuracy: 0.8411
	 Epoch 4500: train accuracy: 0.8411
	

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,psi,0,(stroke_flg | bun_first_NORMAL | sapsi_first_H...,0.837079,0.831461,0.831461,3,0.371429,0.035904
1,psi,1,(stroke_flg | abg_count_NORMAL),0.837079,0.814607,0.814607,2,0.371429,0.025960
2,psi,2,(stroke_flg | abg_count_HIGH | sapsi_first_HIGH),0.837079,0.842697,0.842697,3,0.371429,0.027961
3,psi,3,(stroke_flg | sapsi_first_HIGH),0.842697,0.842697,0.842697,2,0.371429,0.037900
4,psi,4,(stroke_flg | bun_first_NORMAL),0.842697,0.831461,0.831461,2,0.371429,0.023966
5,psi,5,(stroke_flg | bun_first_HIGH | chloride_first_...,0.842697,0.814607,0.814607,3,0.371429,0.029920
6,psi,6,(stroke_flg | bun_first_HIGH | chloride_first_...,0.836158,0.790960,0.790960,3,0.371429,0.034908
7,psi,7,(stroke_flg | bun_first_NORMAL | bun_first_HIGH),0.841808,0.847458,0.847458,3,0.371429,0.030943
8,psi,8,(stroke_flg | bun_first_NORMAL),0.841808,0.807910,0.807910,2,0.371429,0.026928
9,psi,9,(stroke_flg | chloride_first_HIGH | sapsi_firs...,0.841808,0.858757,0.858757,3,0.371429,0.032945


# Decision tree

In [17]:
method = 'tree'
need_pruning = False
relu = False
results_tree = c_to_y(method, need_pruning, relu, verbose=True)
results_tree

Split [1/10]
	 Model's accuracy: 0.8034
	 Class 1 - Global explanation: "(stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & wbc_first_NORMAL <= 0.50 & map_1st_LOW <= 0.50 & bun_first_LOW <= 0.50 & sofa_first_LOW > 0.50) | (stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & wbc_first_NORMAL <= 0.50 & map_1st_LOW <= 0.50 & bun_first_LOW > 0.50 & age_LOW <= 0.50 & hour_icu_intime_HIGH <= 0.50 & po2_first_NORMAL <= 0.50 & hr_1st_HIGH <= 0.50 & sofa_first_NORMAL > 0.50 & potassium_first_NORMAL <= 0.50 & day_icu_intime_num_NORMAL > 0.50 & service_num_HIGH > 0.50 & tco2_first_NORMAL > 0.50 & gender_num > 0.50 & hgb_first_HIGH <= 0.50) | (stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & wbc_first_NORMAL <= 0.50 & map_1st_LOW <= 0.50 & bun_first_LOW > 0.50 & age_LOW <= 0.50 & hour_icu_intime_HIGH <= 0.50 & po2_first_NORMAL <= 0.50 & hr_1st_HIGH <= 0.50 & sofa_first_NORMAL > 0.50 & potassium_first_NORMAL > 0.50 & al

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,tree,0,(stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_...,0.803371,0.803371,0.803371,1363,0.815033,0.004987
1,tree,1,(stroke_flg <= 0.50 & age_HIGH <= 0.50 & sapsi...,0.803371,0.803371,0.803371,1424,0.815033,0.004987
2,tree,2,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & age_N...,0.808989,0.808989,0.808989,1345,0.815033,0.002992
3,tree,3,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & age_N...,0.780899,0.780899,0.780899,1391,0.815033,0.001995
4,tree,4,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & sapsi...,0.786517,0.786517,0.786517,1387,0.815033,0.003991
5,tree,5,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.848315,0.848315,0.848315,1319,0.815033,0.004025
6,tree,6,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.762712,0.762712,0.762712,1271,0.815033,0.004987
7,tree,7,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.813559,0.813559,0.813559,1321,0.815033,0.001995
8,tree,8,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.774011,0.774011,0.774011,1353,0.815033,0.002992
9,tree,9,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & sapsi...,0.774011,0.774011,0.774011,1252,0.815033,0.001995


# BRL

In [11]:
method = 'brl'
need_pruning = False
relu = False
results_brl = c_to_y(method, need_pruning, relu, verbose=True)
results_brl

Split [1/10]
torch.Size([1598, 2])
torch.Size([480, 2])
Labels (1598, 2)
Discretized features
Completed model 1/2!
Completed model 2/2!
Train_acc: 0.9, Val_acc: 0.0
	 Model's accuracy: 85.9551
	 Class 1 - Global explanation: "stroke_flg & weight_first_LOW & ~chf_flg & ~resp_flg & ~abg_count_NORMAL & ~age_LOW & ~age_NORMAL & ~sapsi_first_LOW" - Accuracy: 0.8427
	 Elapsed time 0.005984783172607422
	 Fidelity: "0.8427" - Complexity: "8"
Split [2/10]
torch.Size([1598, 2])
torch.Size([480, 2])
Labels (1598, 2)
Discretized features
Completed model 1/2!
Completed model 2/2!
Train_acc: 0.8, Val_acc: 0.0
	 Model's accuracy: 86.5169
	 Class 1 - Global explanation: "(afib_flg & age_HIGH & map_1st_NORMAL & ~chf_flg & ~resp_flg & ~age_LOW & ~hgb_first_LOW & ~po2_first_NORMAL & ~sapsi_first_LOW & ~wbc_first_HIGH) | (stroke_flg & age_HIGH & map_1st_NORMAL & ~chf_flg & ~afib_flg & ~resp_flg & ~age_LOW & ~hgb_first_LOW & ~po2_first_NORMAL & ~sapsi_first_LOW & ~wbc_first_HIGH)" - Accuracy: 0.8202
	 Elap

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,brl,0,stroke_flg & weight_first_LOW & ~chf_flg & ~re...,85.955056,0.842697,0.842697,8,0.245455,0.005985
1,brl,1,(afib_flg & age_HIGH & map_1st_NORMAL & ~chf_f...,86.516854,0.820225,0.820225,21,0.245455,0.014961
2,brl,2,stroke_flg & weight_first_LOW & ~age_LOW & ~cr...,87.078652,0.870787,0.870787,5,0.245455,0.002993
3,brl,3,stroke_flg & creatinine_first_LOW & weight_fir...,86.516854,0.865169,0.865169,5,0.245455,0.001994
4,brl,4,(bmi_LOW & ~age_NORMAL & ~bun_first_LOW & ~sap...,85.955056,0.865169,0.865169,14,0.245455,0.008975
5,brl,5,stroke_flg & bun_first_LOW & ~age_LOW & ~chlor...,86.516854,0.876404,0.876404,6,0.245455,0.002992
6,brl,6,stroke_flg & resp_flg & creatinine_first_LOW &...,83.615819,0.841808,0.841808,11,0.245455,0.009968
7,brl,7,(stroke_flg & ~age_LOW & ~chloride_first_LOW &...,88.135593,0.881356,0.881356,14,0.245455,0.009974
8,brl,8,(~age_NORMAL & ~bun_first_LOW & ~pco2_first_NO...,88.700565,0.836158,0.836158,24,0.245455,0.015261
9,brl,9,stroke_flg & bun_first_LOW & sapsi_first_NORMA...,87.005650,0.864407,0.864407,7,0.245455,0.004991


# Summary

In [18]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_fidelity', 
        'explanation_complexity', 'elapsed_time', 'explanation_consistency']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

# BRL
df_mean = results_brl[cols].mean()
df_sem = results_brl[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_brl = pd.concat([df_mean, df_sem])
summary_brl.name = 'BRL'

summary = pd.concat([summary_pruning,
                     summary_weights, 
                     summary_psi, 
                     summary_tree,
                     summary_brl], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_mean,explanation_accuracy_mean,explanation_fidelity_mean,explanation_complexity_mean,elapsed_time_mean,explanation_consistency_mean,model_accuracy_sem,explanation_accuracy_sem,explanation_fidelity_sem,explanation_complexity_sem,elapsed_time_sem,explanation_consistency_sem
pruning,0.854161,0.854161,0.854161,2.2,0.064734,0.440000,0.004418,0.004418,0.004418,0.290593,0.005735,0.000000e+00
weights,0.856396,0.327309,0.327309,5.2,0.206950,0.314286,0.005648,0.066790,0.066790,0.904311,0.015183,0.000000e+00
psi,0.840091,0.828261,0.828261,2.6,0.030734,0.371429,0.000894,0.006595,0.006595,0.163299,0.001460,0.000000e+00
tree,0.795575,0.795575,0.795575,1342.6,0.003494,0.815033,0.007968,0.007968,0.007968,16.940550,0.000401,3.700743e-17
BRL,86.599695,0.856418,0.856418,11.5,0.007809,0.245455,0.433616,0.006295,0.006295,2.125245,0.001523,0.000000e+00


In [19]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))